In [10]:
import cv2
import cvzone
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector #We detect the eyes using this library
import pyautogui 
import mediapipe as mp
pyautogui.FAILSAFE = False

In [11]:
cap = cv2.VideoCapture(0)
detector = FaceMeshDetector()
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces = 1)
mpDraw = mp.solutions.drawing_utils

screen_w, screen_h = pyautogui.size()

idList = [159,145,33,133,386,374,362,263] #Eye indices

color = (0,0,255)

counterLeft = 0
counterRight = 0
blinkCounterLeft = 0
blinkCounterRight = 0

rightDistList = []
leftDistList = []


In [12]:
while True:
    success, img = cap.read()
    img, faces = detector.findFaceMesh(img, draw=False)
    img_h, img_w, _ = img.shape 
    results = faceMesh.process(img)
    landmark_points = results.multi_face_landmarks
        
    if faces:
        face = faces[0]
        for id in idList:
            #Marking from the determined indices
            cv2.circle(img, face[id], 2, (0,0,255), cv2.FILLED)
        
        """
        Section for calculating the user's distance to the computer.
        
        """
        pointLeft = face[145] #sol uç
        pointRight = face[374] #sağ uç
        
        """
        Section for control drawing.
        
        """
        # cv2.line(img, pointLeft,pointRight,(0,200,0),3)
        # cv2.circle(img, pointLeft, 5, (255,0,255), cv2.FILLED)
        # cv2.circle(img, pointRight, 5, (255,0,255), cv2.FILLED)
        w, _ = detector.findDistance(pointLeft,pointRight)
        
        W = 6.3
        # d = 50
        # f = (w*d)/W
        # print(f)
        f = 840 
        d = (W*f)/w
        #print(d)
        
        cvzone.putTextRect(img, f"Depth: {int(d)}cm",(face[10][0]-100,face[10][1]-50),scale=2)
        
        """
        Section for measuring the distances of selected points for right-left clicks.
        """

        rightUp = face[159]
        rightDown = face[206]

        leftUp = face[386]
        leftDown = face[287]

        lenghtVerRight, _ = detector.findDistance(rightUp, rightDown)

        cv2.line(img, rightUp, rightDown, (0,255,0), 2)
        # cv2.putText(img, f" Right Eye: {str(int(lenghtVerRight))}", (300,50), cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),3) 


        lenghtVerLeft, _ = detector.findDistance(leftUp, leftDown)

        cv2.line(img, leftUp, leftDown, (0,255,0), 2)
        # cv2.putText(img,f" Left Eye: {str(int(lenghtVerLeft))}", (30,300), cv2.FONT_HERSHEY_PLAIN, 2, (255,0,0),3) 
        
        """
        Section for warnings to keep the user at the correct distance.
        
        Section for detecting clicks with changes in previously selected eye distances.
        
        """
        x, y, w, h = 0, 0, 637, 477  
        if int(d) < 80:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
            close = "GO AWAY"
            cv2.putText(img, str(close), (50,200), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),4)


        elif 80 <= int(d) <=83:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255,0), 5)
            rightDistList.append(lenghtVerRight)
            if len(rightDistList) >5:
                rightDistList.pop(0)
            rightAvg = np.mean(rightDistList)
            #print(rightAvg)

            if 50<rightAvg<51 and counterRight == 0:
                pyautogui.click(button="right")
                blinkCounterRight +=1
                color = (0,255,0)
                counterRight = 1

            if counterRight != 0:
                counterRight += 1
                if counterRight > 10:
                    counterRight =0
                    color =(0,0,255)
            #cvzone.putTextRect(img, f"Right Blink Count: {blinkCounterRight}", (30,100), colorR=color)



            leftDistList.append(lenghtVerLeft)
            if len(leftDistList) >4:
                leftDistList.pop(0)
            leftAvg = np.mean(leftDistList)
            #print(leftAvg)

            if 72<leftAvg<73 and counterLeft == 0:
                pyautogui.click(button="left")
                blinkCounterLeft +=1
                color = (0,255,0)
                counterLeft = 1

            if counterLeft != 0:
                counterLeft += 1
                if counterLeft > 10:
                    counterLeft =0
                    color =(0,0,255)
            #cvzone.putTextRect(img, f"Left Blink Count: {blinkCounterLeft}", (30,400), colorR=color)
        else:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 5)
            close = "GET CLOSER"
            cv2.putText(img, str(close), (50,200), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),4)
            

        """
        Calculation and scaling of the left eye's center to allow the mouse to follow the eye.
        """
        mouse_x = screen_w/img_w * (leftUp[0]+leftDown[0])/2*1.5
        mouse_y = screen_h/img_h * (leftUp[1]+leftDown[1])/2*1.5
        pyautogui.moveTo(mouse_x, mouse_y)

        img = cv2.resize(img, (800,600))  
    
    cv2.imshow("img", img)
    if cv2.waitKey(1) & 0xFF==ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()